In [1]:
import numpy as np
import math
import sys
import time

import numpy.random
from docplex.mp.model import Model
import itertools
import numpy as np
import pandas as pd
from itertools import chain, combinations

In [11]:
#parameter
V = 9
M = 3
#SET
n = [i for i in range(0,V)] # set of node
n = set(n)
m = [k for k in range(0,M)] # set of crew
m = set(m)
t = [[100.0 for i in range(0, V)] for j in range(0,V)]#travel time
o = [[3*60 for i in range(0, V)] for j in range(0,V)] #patching/repairing time
Tm = 8*60
E=[]
t[0][1]=10;t[0][3]=10;t[1][0]=10;t[1][2]=10;t[1][4]=10;t[2][1]=10;t[2][5]=10
t[3][0]=10;t[3][4]=10;t[3][6]=10;t[4][1]=10;t[4][5]=10;t[4][7]=10
t[4][3]=10;t[5][2]=10;t[5][4]=10;t[5][8]=10;t[6][3]=10;t[6][7]=10;t[7][6]=10
t[7][4]=10;t[7][8]=10;t[8][7]=10;t[8][5]=10
for i in n:
    for j in n:
        if t[i][j] <100:
            E.append((i,j)) 
E = set(E)            
Er = [(0,1),(4,5),(6,7)]

In [12]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [10]:
E

{(0, 1),
 (0, 3),
 (1, 0),
 (1, 2),
 (1, 4),
 (2, 1),
 (2, 5),
 (3, 0),
 (3, 4),
 (3, 6),
 (4, 1),
 (4, 3),
 (4, 5),
 (4, 7),
 (5, 2),
 (5, 4),
 (5, 8),
 (6, 3),
 (6, 7),
 (7, 4),
 (7, 6),
 (7, 8),
 (8, 5),
 (8, 7)}

In [25]:
mdl = Model('dispatch_pothole')
x = mdl.integer_var_dict([(i,j,k) for i in n for j in n for k in m], name='x')

In [22]:
mdl.minimize(mdl.sum(x[i,j,k]*t[i][j] for i in n for j in n for k in m))
mdl.add_constraints(mdl.sum(x[i,j,k] for j in n)-mdl.sum(x[j,i,k] for j in n) == 0 for i in n for k in m)
#mdl.add_constraint(mdl.sum(x[0,4,k]+x[4,0,k] for k in m) >=1)
for e in E:
    mdl.add_constraint(mdl.sum(x[e[0],e[1],k]+x[e[1],e[0],k] for k in m) >= 1)
mdl.add_constraints(mdl.sum(x[i,j,k]*t[i][j] for i in n for j in n) <= Tm for k in m)  
mdl.add_constraints(mdl.sum(x[0,j,k] for j in n) == 1 for k in m)
mdl.add_constraints(mdl.sum(x[j,0,k] for j in n) == 1 for k in m)

[docplex.mp.LinearConstraint[](x_0_0_0+x_1_0_0+x_2_0_0+x_3_0_0+x_4_0_0+x_5_0_0+x_6_0_0+x_7_0_0+x_8_0_0,EQ,1),
 docplex.mp.LinearConstraint[](x_0_0_1+x_1_0_1+x_2_0_1+x_3_0_1+x_4_0_1+x_5_0_1+x_6_0_1+x_7_0_1+x_8_0_1,EQ,1),
 docplex.mp.LinearConstraint[](x_0_0_2+x_1_0_2+x_2_0_2+x_3_0_2+x_4_0_2+x_5_0_2+x_6_0_2+x_7_0_2+x_8_0_2,EQ,1)]